## scrapy的概念和流程

###  什么是scrapy？

文档地址：http://scrapy-chs.readthedocs.io/zh_CN/1.0/intro/overview.html

Scrapy 使用了Twisted['twɪstɪd]异步网络框架，可以加快我们的下载速度。

**Scrapy是一个为了爬取网站数据，提取结构性数据而编写的应用框架**，我们只需要实现少量的代码，就能够快速的抓取。

###  异步和非阻塞的区别

前面我们说Twisted是一个异步的网络框架，经常我们也听到一个词语叫做非阻塞，那么他们有什么区别呢？

<img src="http://39.106.195.80:8888/group1/M00/00/00/wKgCa1_R_WKAG4qRAAGYaQ58Aus069.png" width=80%>


### scrapy的工作流程

#### 爬虫的一般流程

<img src="http://39.106.195.80:8888/group1/M00/00/00/wKgCa1_R-WSAXkrYAAFWmRF8qQ0555.png" width=60%>

-------
也可以改为

<img src="http://39.106.195.80:8888/group1/M00/00/00/wKgCa1_R-cCACWUwAAICveq-Khs001.png" width=70%>

------
#### scrapy的流程：

<img src="http://39.106.195.80:8888/group1/M00/00/00/wKgCa1_R-sWASczOAAOYF7KA3xo137.png" width=80%>

## scrapy的入门使用


###  scrapy项目实现流程

1. 创建一个scrapy项目:scrapy startproject mySpider
2. 生成一个爬虫:scrapy genspider baidu "baidu.com"
3. 提取数据:完善spider，使用xpath等方法
4. 保存数据:pipeline中保存数据

### 创建scrapy项目

安装scrapy命令：pip install scrapy

创建scrapy项目的命令：scrapy startproject +<项目名字>

- 示例：scrapy startproject myspider




###  创建爬虫

命令：**在项目路径下执行**:scrapy genspider +<爬虫名字> + <允许爬取的域名>

示例：

```
cd myspider
scrapy genspider baidu baidu.com
```



###  完善spider

完善spider即通过方法进行数据的提取等操作


>注意：
>
    1. response.xpath方法的返回结果是一个类似list的类型，其中包含的是selector对象，操作和列表一样，但是有一些额外的方法
    2. extract() 返回一个包含有字符串的列表
    3. extract_first() 返回列表中的第一个字符串，列表为空没有返回None
    4. spider中的parse方法必须有
    5. 需要抓取的url地址必须属于allowed_domains,但是start_urls中的url地址没有这个限制
    6. 启动爬虫的时候注意启动的位置，是在项目路径下启动

###  利用管道pipeline来处理(保存)数据

#### 对爬虫进行修改完善

在爬虫文件中parse()函数中最后添加`yield item`

#### 思考：为什么要使用yield？

1. 让整个函数变成一个生成器，有什么好处呢？
2. 遍历这个函数的返回值的时候，挨个把数据读到内存，不会造成内存的瞬间占用过高

**注意：yield能够传递的对象只能是：BaseItem,Request,dict,None**


###  修改pipelines.py文件

```python
class EgPipeline(object):
    # 爬虫文件中提取数据的方法每yield一次item，就会运行一次
    # 该方法为固定名称函数
    def process_item(self, item, spider):
        print(item)
```



### 在settings.py设置开启pipeline

```python
ITEM_PIPELINES = {
    'myspider.pipelines.EgPipeline': 400
}
```



###  运行scrapy

命令：在项目目录下执行scrapy crawl +<爬虫名字>

示例：scrapy crawl baidu


## scrapy发送翻页请求



### 翻页请求的思路

对于要提取如下图中所有页面上的数据该怎么办？
<img src="http://39.106.195.80:8888/group1/M00/00/00/wKgCa1_RyRqAFdm0AAAmWwMQnMg178.png" width=40%>
<img src="http://39.106.195.80:8888/group1/M00/00/00/wKgCa1_R0E-ACysaAAAv0mNCa1s440.png" width=60%>

回顾requests模块是如何实现翻页请求的：

1. 找到下一页的URL地址
2. 调用requests.get(url)

scrapy实现翻页的思路：

1. 找到下一页的url地址
2. 构造url地址的请求，传递给引擎


###  scrapy实现翻页请求

#### 实现方法

1. 确定url地址
2. 构造请求，scrapy.Request(url,callback)
   - callback：指定解析函数名称，表示该请求返回的响应使用哪一个函数进行解析
3. 把请求交给引擎：yield scrapy.Request(url,callback)


####  实战：腾讯招聘爬虫

> 通过爬取安居客北京二手房信息,学习如何实现翻页请求
>
> 地址：https://beijing.anjuke.com/sale/?from=navigation



##### 思路分析：

1. 获取首页的数据
2. 寻找下一页的地址，进行翻页，获取数据



##### 注意：

1. 可以在settings中设置ROBOTS协议

   ```python
   # False表示忽略网站的robots.txt协议，默认为True
   ROBOTSTXT_OBEY = False
   ```

2. 可以在settings中设置User-Agent：

   ```python
   # scrapy发送的每一个请求的默认UA都是设置的这个User-Agent
   USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'
   ```



####  代码实现

在爬虫文件的parse方法中：

```python
......
    # 提取下一页的href并拼接url
    next_url = response.xpath('//a[text()="下一页"]/@href').extract_first()
    # 判断是否是最后一页
    if response.xpath('//a[text()="下一页"]/@href').extract_first() != 'javascript:;':
        # 构造scrapy.Request对象，并yield给引擎
        # 利用callback参数指定该Request对象之后获取的响应用哪个函数进行解析
        # 利用meta参数将本函数中提取的数据传递给callback指定的函数
        # 注意这里是yield
        yield scrapy.Request(next_url, callback=self.parse)
......
```



####  scrapy.Request的更多参数

```python
scrapy.Request(url[,callback,method="GET",headers,body,cookies,meta,dont_filter=False])
```


##### 参数解释

1. 中括号中的参数为可选参数
2. callback：表示当前的url的响应交给哪个函数去处理
3. meta：实现数据在不同的解析函数中传递，meta默认带有部分数据，比如下载延迟，请求深度等
4. dont_filter:默认为False，会过滤请求的url地址，即请求过的url地址不会继续被请求，对需要重复请求的url地址可以把它设置为Ture，比如贴吧的翻页请求，页面的数据总是在变化;start_urls中的地址会被反复请求，否则程序不会启动
5. method：指定POST或GET请求
6. headers：接收一个字典，其中不包括cookies
7. cookies：接收一个字典，专门放置cookies
8. body：接收一个字典，为POST的数据



##### meta参数的使用

> meta的形式:字典
>
> meta的作用：meta可以实现数据在不同的解析函数中的传递

在爬虫文件的parse方法中，提取详情页增加之前callback指定的parse_detail函数：

```python
def parse(self,response):
    ...
    yield scrapy.Request(detail_url, callback=self.parse_detail,meta={"item":item})
...

def parse_detail(self,response):
    #获取之前传入的item
    item = resposne.meta["item"]
```



##### 特别注意

1. meta参数是一个字典
2. meta字典中有一个固定的键`proxy`，表示代理ip



### item的使用

####  Item能够做什么

1. 定义item即提前规划好哪些字段需要抓取，scrapy.Field()仅仅是提前占坑，通过item.py能够让别人清楚自己的爬虫是在抓取什么，同时定义好哪些字段是需要抓取的，没有定义的字段不能使用，防止手误
2. 在python大多数框架中，大多数框架都会自定义自己的数据类型(在python自带的数据结构基础上进行封装)，目的是增加功能，增加自定义异常



####  定义Item

在items.py文件中定义要提取的字段：

```python
class AnjukeItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    img_src = scrapy.Field()
    title = scrapy.Field()
    detail_list = scrapy.Field()
    address = scrapy.Field()
    price = scrapy.Field()
    avg_price = scrapy.Field()
```


####  使用Item

Item使用之前需要先导入并且实例化，之后的使用方法和使用字典相同

修改爬虫文件itcast.py：

```python
from Tencent.items import AnjukeItem # 导入Item，注意路径
...
    def parse_detail(self, response):
        # 进入详细页面进行爬取 

        print(item)
```



##### 注意：

python中的导入路径要诀：从哪里开始运行，就从哪里开始导入

https://careers.tencent.com/search.html

## scrapy的深入使用




### 了解scrapyShell

scrapy shell是scrapy提供的一个终端工具，能够通过它查看scrapy中对象的属性和方法，以及测试xpath

使用方法：

```linux
scrapy shell http://www.baidu.com
```

在终端输入上述命令后，能够进入python的交互式终端，此时可以使用：

- response.xpath()：直接测试xpath规则是否正确
- response.url：当前响应的url地址
- response.request.url：当前响应对应的请求的url地址
- response.headers：响应头
- response.body：响应体，也就是html代码，默认是byte类型
- response.requests.headers：当前响应的请求头


###  settings.py中的设置信息

#### 为什么项目中需要配置文件

    在配置文件中存放一些公共变量

#### 配置文件中的变量使用方法

    1. 变量名一般全部大写
    2. 导入即可使用


#### settings.py中的重点字段和含义

- USER_AGENT 设置ua
- ROBOTSTXT_OBEY 是否遵守robots协议，默认是遵守
- CONCURRENT_REQUESTS 设置并发请求的数量，默认是16个
- DOWNLOAD_DELAY 下载延迟，默认无延迟
- COOKIES_ENABLED 是否开启cookie，即每次请求带上前一次的cookie，默认是开启的
- DEFAULT_REQUEST_HEADERS 设置默认请求头，这里加入了USER_AGENT将不起作用
- SPIDER_MIDDLEWARES 爬虫中间件，设置过程和管道相同
- DOWNLOADER_MIDDLEWARES 下载中间件
- LOG_LEVEL 控制终端输出信息的log级别，终端默认显示的是debug级别的log信息
  - LOG_LEVEL = "WARNING"
- LOG_FILE 设置log日志文件的保存路径，如果设置该参数，终端将不再显示信息
  - LOG_FILE = "./test.log"




###  pipeline管道的使用

#### pipeline中常用的方法：

1. process_item(self,item,spider):实现对item数据的处理
2. open_spider(self, spider): 在爬虫开启的时候仅执行一次
3. close_spider(self, spider): 在爬虫关闭的时候仅执行一次



####  管道文件的修改

在pipelines.py代码中完善

```python
import json

class AnjukePipeline(object):
    def open_spider(self, spider):  # 在爬虫开启的时候仅执行一次
        if spider.name == 'itcast':
            self.f = open('json.txt', 'a', encoding='utf-8')

    def close_spider(self, spider):  # 在爬虫关闭的时候仅执行一次
        if spider.name == 'itcast':
            self.f.close()

    def process_item(self, item, spider):
        if spider.name == 'itcast':
            self.f.write(json.dumps(dict(item), ensure_ascii=False, indent=2) + ',\n')
        return item  # 不return的情况下，另一个权重较低的pipeline将不会获得item
```


#### 开启管道

在settings.py设置开启pipeline

```python
......
ITEM_PIPELINES = {
    'myspider.pipelines.AnjukePipeline': 400, # 400表示权重
    'myspider.pipelines.AnjukePipeline2': 500,
}
......
```

**思考：pipeline在settings中能够开启多个，为什么需要开启多个？**

1. 不同的pipeline可以处理不同爬虫的数据，通过spider.name属性来区分
2. 不同的pipeline能够对一个或多个爬虫进行不同的数据处理的操作，比如一个进行数据清洗，一个进行数据的保存
3. 同一个管道类也可以处理不同爬虫的数据，通过spider.name属性来区分



#### pipeline使用注意点

1. 使用之前需要在settings中开启
2. pipeline在setting中键表示位置(即pipeline在项目中的位置可以自定义)，值表示距离引擎的远近，越近数据会越先经过：**权重值小的优先执行**
3. 有多个pipeline的时候，process_item的方法必须return item,否则后一个pipeline取到的数据为None值
4. pipeline中process_item的方法必须有，否则item没有办法接受和处理
5. process_item方法接受item和spider，其中spider表示当前传递item过来的spider
6. open_spider(spider) :能够在爬虫开启的时候执行一次
7. close_spider(spider) :能够在爬虫关闭的时候执行一次
8. 上述俩个方法经常用于爬虫和数据库的交互，在爬虫开启的时候建立和数据库的连接，在爬虫关闭的时候断开和数据库的连接


## scrapy的crawlspider爬虫



### crawlspider是什么

> 回顾之前的代码中，我们有很大一部分时间在寻找下一页的url地址或者是内容的url地址上面，这个过程能更简单一些么？

#### 思路：

1. 从response中提取所有的满足规则的url地址
2. 自动的构造自己requests请求，发送给引擎

对应的**crawlspider就可以实现上述需求，能够匹配满足条件的url地址，组装成Reuqest对象后自动发送给引擎，同时能够指定callback函数**

**即：crawlspider爬虫可以按照规则自动获取连接**



###  创建crawlspider爬虫并观察爬虫内的默认内容

#### 创建crawlspider爬虫：

```linux
scrapy genspider -t crawl tencent hr.tencent.com
```


#### spider中默认生成的内容如下：

```python
class TencentSpider(CrawlSpider):
    name = 'tencent'
    allowed_domains = ['careers.tencent.com']
    start_urls = ['https://careers.tencent.com/']

    rules = (
        Rule(LinkExtractor(allow=r'Items/'), callback='parse_item', follow=True),
    )

    def parse_item(self, response):
       pass
```


#### 观察跟普通的scrapy.spider的区别

在crawlspider爬虫中，没有parse函数

##### 重点在rules中：

1. rules是一个元组或者是列表，包含的是Rule对象
2. Rule表示规则，其中包含LinkExtractor,callback和follow等参数
3. LinkExtractor:连接提取器，可以通过正则或者是xpath来进行url地址的匹配
4. callback :表示经过连接提取器提取出来的url地址响应的回调函数，可以没有，没有表示响应不会进行回调函数的处理
5. follow：连接提取器提取的url地址对应的响应是否还会继续被rules中的规则进行提取，True表示会，Flase表示不会


###  crawlspider腾讯招聘爬虫

#### 思路分析：

1. 定义一个规则，来进行列表页翻页，follow需要设置为True
2. 定义一个规则，实现从列表页进入详情页，并且指定回调函数
3. 在详情页提取数据

###### 注意：连接提取器LinkExtractor中的allow对应的正则表达式匹配的是href属性的值



### crawlspider使用的注意点：

1. 除了用命令`scrapy genspider -t crawl <爬虫名> <allowed_domail>`创建一个crawlspider的模板，页可以手动创建
2. crawlspider中不能再有以parse为名的数据提取方法，该方法被crawlspider用来实现基础url提取等功能
3. Rule对象中LinkExtractor为固定参数，其他callback、follow为可选参数
4. 不指定callback且follow为True的情况下，满足rules中规则的url还会被继续提取和请求
5. 如果一个被提取的url满足多个Rule，那么会从rules中选择一个满足匹配条件的Rule执行


### 了解crawlspider其他知识点

- 链接提取器LinkExtractor的更多常见参数
  - allow: 满足括号中的're'表达式的url会被提取，如果为空，则全部匹配
  - deny: 满足括号中的're'表达式的url不会被提取，优先级高于allow
  - allow_domains: 会被提取的链接的domains(url范围)，如：`['baidu.com']`
  - deny_domains: 不会被提取的链接的domains(url范围)
  - **restrict_xpaths: 使用xpath规则进行匹配，和allow共同过滤url，即xpath满足的范围内的url地址会被提取**，如：`restrict_xpaths='//div[@class="pagenav"]'`
- Rule常见参数
  - LinkExtractor: 链接提取器，可以通过正则或者是xpath来进行url地址的匹配
  - callback: 表示经过连接提取器提取出来的url地址响应的回调函数，可以没有，没有表示响应不会进行回调函数的处理
  - follow: 连接提取器提取的url地址对应的响应是否还会继续被rules中的规则进行提取，默认True表示会，Flase表示不会
  - process_links: 当链接提取器LinkExtractor获取到链接列表的时候调用该参数指定的方法，这个自定义方法可以用来过滤url，且这个方法执行后才会执行callback指定的方法


## scrapy中间件

### 学习目标：

1. 应用 scrapy中使用间件使用随机UA的方法
2. 了解 scrapy中使用代理ip的的方法



###  scrapy中间件的分类和作用

#### scrapy中间件的分类

根据scrapy运行流程中所在位置不同分为：

1. 下载中间件
2. 爬虫中间件

#### scrapy中间的作用

1. 主要功能是在爬虫运行过程中进行一些处理，如对非200响应的重试（重新构造Request对象yield给引擎）
2. 也可以对header以及cookie进行更换和处理
3. 其他根据业务需求实现响应的功能

但在scrapy默认的情况下 两种中间件都在middlewares.py一个文件中

爬虫中间件使用方法和下载中间件相同，常用下载中间件


###  下载中间件的使用方法：

> 编写一个Downloader Middlewares和我们编写一个pipeline一样，定义一个类，然后在setting中开启

Downloader Middlewares默认的方法：

- process_request(self, request, spider)：
  1. 当每个request通过下载中间件时，该方法被调用。
  2. 返回None值：没有return也是返回None，该request对象传递给下载器，或通过引擎传递给其他权重低的process_request方法
  3. 返回Response对象：不再请求，把response返回给引擎
  4. 返回Request对象：把request对象通过引擎交给调度器，此时将不通过其他权重低的process_request方法
- process_response(self, request, response, spider)：
  1. 当下载器完成http请求，传递响应给引擎的时候调用
  2. 返回Resposne：通过引擎交给爬虫处理或交给权重更低的其他下载中间件的process_response方法
  3. 返回Request对象：通过引擎交给调取器继续请求，此时将不通过其他权重低的process_request方法
- 在settings.py中配置开启中间件，权重值越小越优先执行



###  定义实现随机User-Agent的下载中间件

#### 在middlewares.py中完善代码

```python
import random
from Tencent.settings import USER_AGENTS_LIST # 注意导入路径,请忽视pycharm的错误提示

class UserAgentMiddleware(object):
    def process_request(self, request, spider):
        user_agent = random.choice(USER_AGENTS_LIST)
        request.headers['User-Agent'] = user_agent

class CheckUA:
    def process_response(self,request,response,spider):
        print(request.headers['User-Agent'])
        return response # 不能少！
```



#### 在settings中设置开启自定义的下载中间件，设置方法同管道

```python
DOWNLOADER_MIDDLEWARES = {
   'Tencent.middlewares.UserAgentMiddleware': 543, # 543是权重值
   'Tencent.middlewares.CheckUA': 600, # 先执行543权重的中间件，再执行600的中间件
}
```



#### 在settings中添加UA的列表

```python
USER_AGENTS_LIST = [
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5"
]
```


### 代理ip的使用

#### 思路分析

1. 代理添加的位置：request.meta中增加`proxy`字段

2. 获取一个代理ip，赋值给

   ```
   request.meta['proxy']
   ```

   - 代理池中随机选择代理ip
   - 代理ip的webapi发送请求获取一个代理ip




免费代理ip：

```python
class ProxyMiddleware(object):
    def process_request(self,request,spider):
        # proxies可以在settings.py中，也可以来源于代理ip的webapi
        # proxy = random.choice(proxies) 

        # 免费的会失效，报 111 connection refused 信息！重找一个代理ip再试
        proxy = 'https://1.71.188.37:3128' 

        request.meta['proxy'] = proxy
        return None # 可以不写return
```



#### 检测代理ip是否可用

在使用了代理ip的情况下可以在下载中间件的process_response()方法中处理代理ip的使用情况，如果该代理ip不能使用可以替换其他代理ip

```python
class ProxyMiddleware(object):
    ......
    def process_response(self, request, response, spider):
        if response.status != '200' and response.status != '302' and response.status != '301':
            #此时对代理ip进行操作，比如删除
            return request
```



#### 在settings.py中开启该中间件


## scrapy模拟登陆

### 学习目标：

1. 应用 scrapy直接携带cookie模拟登陆的方法
2. 应用 scrapy.FormRequest()发送post请求进行登陆
3. 应用 scrapy.FormRequest.from_response()发送表单请求

### 回顾之前的模拟登陆的方法

#### requests模块是如何实现模拟登陆的？

1. 直接携带cookies请求页面
2. 找url地址，发送post请求存储cookie

#### selenium是如何模拟登陆的？

1. 找到对应的input标签，输入文本点击登陆

#### scrapy有三种方法模拟登陆

1. 直接携带cookies
2. 找url地址，发送post请求存储cookie
3. 找到对应的form表单，自动解析input标签，自动解析post请求的url地址，自动带上数据，自动发送请求


###  scrapy携带cookies直接获取需要登陆后的页面

#### 应用场景

1. cookie过期时间很长，常见于一些不规范的网站
2. 能在cookie过期之前把all数据拿到
3. 配合其他程序使用，比如其使用selenium把登陆之后的cookie获取到保存到本地，scrapy发送请求之前先读取本地cookie



#### 实现：重构scrapy的starte_rquests方法

scrapy中start_url是通过start_requests来进行处理的，其实现代码如下

```python
def start_requests(self):
    cls = self.__class__
    if method_is_overridden(cls, Spider, 'make_requests_from_url'):
        warnings.warn(
            "Spider.make_requests_from_url method is deprecated; it "
            "won't be called in future Scrapy releases. Please "
            "override Spider.start_requests method instead (see %s.%s)." % (
                cls.__module__, cls.__name__
            ),
        )
        for url in self.start_urls:
            yield self.make_requests_from_url(url)
    else:
        for url in self.start_urls:
            yield Request(url, dont_filter=True)
```

**所以对应的，如果start_url地址中的url是需要登录后才能访问的url地址，则需要重写start_request方法并在其中手动添加上cookie**


#### 携带cookies登陆github

```python
import scrapy
import re

class Login1Spider(scrapy.Spider):
    name = 'login1'
    allowed_domains = ['github.com']
    start_urls = ['https://github.com/NoobPythoner'] # 这是一个需要登陆以后才能访问的页面

    def start_requests(self): # 重构start_requests方法
        # 这个cookies_str是抓包获取的
        cookies_str = '...' # 抓包获取
        # 将cookies_str转换为cookies_dict
        cookies_dict = {i.split('=')[0]:i.split('=')[1] for i in cookies_str.split('; ')}
        yield scrapy.Request(
            self.start_urls[0],
            callback=self.parse,
            cookies=cookies_dict
        )

    def parse(self, response): # 通过正则表达式匹配用户名来验证是否登陆成功
        # 正则匹配的是github的用户名
        result_list = re.findall(r'noobpythoner|NoobPythoner', response.body.decode()) 
        print(result_list)
        pass
```

##### 注意：

1. scrapy中cookie不能够放在headers中，在构造请求的时候有专门的cookies参数，能够接受字典形式的coookie
2. 在setting中设置ROBOTS协议、USER_AGENT



### scrapy.FormRequest发送post请求

> 我们知道可以通过scrapy.Request()指定method、body参数来发送post请求；那么也可以使用scrapy.FormRequest()来发送post请求

#### scrapy.FormRequest()的使用

通过scrapy.FormRequest能够发送post请求，同时需要添加fromdata参数作为请求体，以及callback

```python
yield scrapy.FormRequest(
    "https://github.com/session",
    formdata={
        "authenticity_token":authenticity_token,
        "utf8":utf8,
        "commit":commit,
        "login":"noobpythoner",
        "password":"zhoudawei123"
    },
    callback=self.parse_login
)
```




### 使用scrapy.FormRequest()登陆github

#### 思路分析

1. 找到post的url地址：点击登录按钮进行抓包，然后定位url地址为https://github.com/session
2. 找到请求体的规律：分析post请求的请求体，其中包含的参数均在前一次的响应中
3. 否登录成功：通过请求个人主页，观察是否包含用户名

####  代码实现如下：

```python
import scrapy
import re

class Login2Spider(scrapy.Spider):
   name = 'login2'
   allowed_domains = ['github.com']
   start_urls = ['https://github.com/login']

   def parse(self, response):
       authenticity_token = response.xpath("//input[@name='authenticity_token']/@value").extract_first()
       utf8 = response.xpath("//input[@name='utf8']/@value").extract_first()
       commit = response.xpath("//input[@name='commit']/@value").extract_first()

        #构造POST请求，传递给引擎
       yield scrapy.FormRequest(
           "https://github.com/session",
           formdata={
               "authenticity_token":authenticity_token,
               "utf8":utf8,
               "commit":commit,
               "login":"1234566",
               "password":"***"
           },
           callback=self.parse_login
       )

   def parse_login(self,response):
       ret = re.findall(r"noobpythoner|NoobPythoner",response.text)
       print(ret)
```


###  scrapy自动提交表单

####  scrapy.Formrequest.from_response

它能够自动的从响应中寻找form表单，然后把formdata中的数据提交到action对应的url地址中

```python
yield scrapy.FormRequest.from_response(
    response, # 传入response对象,自动解析
    # 可以通过xpath来定位form表单,当前页只有一个form表单时,将会自动定位
    formxpath='//*[@id="login"]/form',  # 可以不写
    formdata={'login': 'noobpythoner', 'password': '***'},
    callback=self.parse_login
)
```

####  使用scrapy.Formrequest.from_response登陆github

```python
import scrapy
import re

class Login3Spider(scrapy.Spider):
    name = 'login3'
    allowed_domains = ['github.com']
    start_urls = ['https://github.com/login']

    def parse(self, response):
        yield scrapy.FormRequest.from_response(
            response, # 传入response对象,自动解析
            # 可以通过xpath来定位form表单,当前页只有一个form表单时,将会自动定位
            formxpath='//*[@id="login"]/form', 
            formdata={'login': 'noobpythoner', 'password': 'zhoudawei123'},
            callback=self.parse_login
        )

    def parse_login(self,response):
        ret = re.findall(r"noobpythoner|NoobPythoner", response.text)
        print(ret)
```

## scrapy内置媒体（图片和文件）下载方式

scrapy为下载的item中包含的文件提供了一个可重用的item pipeline（scrapy.pipelines.media.MediaPipeline），这些Pipeline有些共同的方法和结构。

MediaPipeline共同实现了以下特性：

　　（1）避免重新下载最近已经下载过的数据

　　（2）指定存储的位置和方式

ImagesPipeline还提供了额外的特性：

　　（1）将所有下载的图片转换成通用的格式（JPG）和模式（RGB）

　　（2）缩略图生成

　　（3）检查图像的宽/高，确保它们满足最小限制

MediaPipeline去重的方式：

　　（1）MediaPipeline会为当前安排好的要下载的图片保留一个**内部队列**，并将那些到达的包含相同图片的项目连接到该队列中，避免多次下载几个item共享的同一图片